In [6]:
# %%
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 定义Add & Norm层
class AddNorm(nn.Module):
    def __init__(self, hidden_dim):
        super(AddNorm, self).__init__()
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, residual):
        return self.norm(x + residual)

# 定义Feed Forward层
class FeedForward(nn.Module):
    def __init__(self, hidden_dim, ff_dim):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(hidden_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# 定义Multi-Head Attention层
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        Q = self.query(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        energy = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim=-1)

        x = torch.matmul(attention, V).transpose(1, 2).contiguous().view(batch_size, -1, self.hidden_dim)
        x = self.fc(x)

        return x

# 定义Encoder层
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.ff = FeedForward(hidden_dim, ff_dim)
        self.norm1 = AddNorm(hidden_dim)
        self.norm2 = AddNorm(hidden_dim)

    def forward(self, x, mask=None):
        residual = x
        x = self.self_attn(x, x, x, mask)
        x = self.norm1(x, residual)

        residual = x
        x = self.ff(x)
        x = self.norm2(x, residual)

        return x

# 定义Decoder层
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.cross_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.ff = FeedForward(hidden_dim, ff_dim)
        self.norm1 = AddNorm(hidden_dim)
        self.norm2 = AddNorm(hidden_dim)
        self.norm3 = AddNorm(hidden_dim)

    def forward(self, x, enc_outputs, src_mask=None, tgt_mask=None):
        residual = x
        x = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x, residual)

        residual = x
        x = self.cross_attn(x, enc_outputs, enc_outputs, src_mask)
        x = self.norm2(x, residual)

        residual = x
        x = self.ff(x)
        x = self.norm3(x, residual)

        return x
class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, hidden_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-np.log(10000.0) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x
class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, num_layers, ff_dim):
        super(Transformer, self).__init__()
        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        self.output_embedding = nn.Linear(hidden_dim, output_dim)

        self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])

    def forward(self, src, tgt_mask=None):
        src_embed = self.input_embedding(src)

        enc_outputs = src_embed
        for enc_layer in self.encoder_layers:
            enc_outputs = enc_layer(enc_outputs)

        dec_outputs = enc_outputs
        for dec_layer in self.decoder_layers:
            dec_outputs = dec_layer(dec_outputs, enc_outputs, tgt_mask=tgt_mask)

        outputs = self.output_embedding(dec_outputs[:, -1, :])
        return outputs
'''
# 定义Transformer模型
class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, num_layers, ff_dim):
        super(Transformer, self).__init__()
        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_encoding = PositionalEncoding(hidden_dim)
        self.output_embedding = nn.Linear(hidden_dim, output_dim)

        self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])

    def forward(self, src, tgt_mask=None):
        src_embed = self.input_embedding(src)
        src_embed = self.pos_encoding(src_embed)

        enc_outputs = src_embed
        for enc_layer in self.encoder_layers:
            enc_outputs = enc_layer(enc_outputs)

        dec_outputs = enc_outputs
        for dec_layer in self.decoder_layers:
            dec_outputs = dec_layer(dec_outputs, enc_outputs, tgt_mask=tgt_mask)

        outputs = self.output_embedding(dec_outputs[:, -1, :])
        return outputs'''

# %%
import pandas as pd
data = pd.read_csv('tmp2.csv')
#data_stock=data[data.name=='万科A']

features = ['company_id','open', 'close', 'high', 'low', 'volume', 'money_netflow', 'money_inflow', 'money_outflow',
       'net_inflow_rate', 'list_sector', 'CPI', '无风险利率',
       'total_market_cap', 'float_market_cap', 'pe_ttm', 'pb',
       'dividend_yield_ratio', 'major_id', 'minor_id',
       'change_ratio']
data = data[features]

# %%
import numpy as np

def create_dataset(data, look_back=5):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), :-1])
        Y.append(data[i + look_back, -1])
    return np.array(X), np.array(Y)

# %%
def create_all_dataset(data, look_back=5):
    X, Y = [], []
    # 划分
    for i in range(len(data.company_id.unique())):
        minidata=data[data.company_id==i]
        x, y = create_dataset(minidata.values, look_back) # array合并
        X.append(x)
        Y.append(y)
    return np.concatenate(X), np.concatenate(Y)

# %%
from sklearn.preprocessing import StandardScaler


# 创建 StandardScaler 对象
scaler = StandardScaler()

# 对数据进行归一化处理

scale_need_cols = [ 'open', 'close', 'high', 'low', 'volume', 'money_netflow', 'money_inflow', 'money_outflow',
       'net_inflow_rate', 'CPI', '无风险利率',
       'total_market_cap', 'float_market_cap', 'pe_ttm', 'pb',
       'dividend_yield_ratio',
       'change_ratio']

data.loc[:, scale_need_cols] = scaler.fit_transform(data[scale_need_cols])

# %%
look_back = 5
X, Y = create_all_dataset(data, look_back)

# %%
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#设置训练设备
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
# 设置模型参数
input_dim = 20
hidden_dim = 128
output_dim = 1
num_heads = 8
num_layers = 6
ff_dim = 512

# 创建模型实例
model = Transformer(input_dim, hidden_dim, output_dim, num_heads, num_layers, ff_dim)
model.to(device)

# %%
def directional_accuracy(actual, predicted):
    return np.mean(np.sign(actual) == np.sign(predicted))

# %%
import os
def train_model(model, optimizer, criterion, train_src, train_tgt, val_src, val_tgt, num_epochs, batch_size, eval_interval=100, checkpoint_interval=100, checkpoint_dir='checkpoints'):
    os.makedirs(checkpoint_dir, exist_ok=True)  # 创建checkpoint目录
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # 训练模式
        model.train()
        epoch_train_loss = 0
        for i in range(0, len(train_src), batch_size):
            batch_src = train_src[i:i+batch_size]
            batch_tgt = train_tgt[i:i+batch_size]
            optimizer.zero_grad()
            outputs = model(batch_src)
            loss = criterion(outputs.view(-1), batch_tgt.view(-1))
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        train_losses.append(epoch_train_loss / (len(train_src) / batch_size))
        
        # 每隔eval_interval个epoch进行一次评估
        if (epoch + 1) % eval_interval == 0:
            # 评估模式
            model.eval()
            model.to('cpu')  # 将模型移动到CPU
            with torch.no_grad():
                train_outputs = model(train_src.to('cpu'))  # 将输入数据移动到CPU
                val_outputs = model(val_src.to('cpu'))  # 将输入数据移动到CPU 
                train_loss = criterion(train_outputs.view(-1), train_tgt.view(-1).to('cpu'))  # 将目标数据移动到CPU
                val_loss = criterion(val_outputs.view(-1), val_tgt.view(-1).to('cpu'))  # 将目标数据移动到CPU
                train_mae = torch.mean(torch.abs(train_tgt.to('cpu') - train_outputs.view(-1)))  # 将目标数据移动到CPU
                val_mae = torch.mean(torch.abs(val_tgt.to('cpu') - val_outputs.view(-1)))  # 将目标数据移动到CPU
                train_rmse = torch.sqrt(torch.mean((train_tgt.to('cpu') - train_outputs.view(-1))**2))  # 将目标数据移动到CPU
                val_rmse = torch.sqrt(torch.mean((val_tgt.to('cpu') - val_outputs.view(-1))**2))  # 将目标数据移动到CPU
                train_da = directional_accuracy(train_tgt.to('cpu'), train_outputs.view(-1))  # 将目标数据移动到CPU
                val_da = directional_accuracy(val_tgt.to('cpu'), val_outputs.view(-1))  # 将目标数据移动到CPU
            model.to(device)  # 将模型移动回GPU
            val_losses.append(val_loss.item())
            
            # 如果当前epoch的验证损失是最好的，就保存当前模型
            if val_loss.item() < best_val_loss:
                best_val_loss = val_loss.item()
                torch.save(model.state_dict(), f"{checkpoint_dir}/best_model.pt")
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.8f}, Train MAE: {train_mae.item():.8f}, Train RMSE: {train_rmse.item():.8f}, Train DA: {train_da:.8f}, Val Loss: {val_loss.item():.8f}, Val MAE: {val_mae.item():.8f}, Val RMSE: {val_rmse.item():.8f}, Val DA: {val_da:.8f}")
        
        # 每隔checkpoint_interval个epoch保存一次模型
        if (epoch + 1) % checkpoint_interval == 0:
            torch.save(model.state_dict(), f"{checkpoint_dir}/model_epoch_{epoch+1}.pt")

# %%

# 准备训练和验证数据
train_src_data = X_train
train_tgt_data = Y_train
val_src_data = X_test
val_tgt_data = Y_test

train_src_tensor = torch.tensor(train_src_data,dtype=torch.float32,device=device)
#train_src_tensor = train_src_data.clone().detach().to(device)

train_tgt_tensor = torch.tensor(train_tgt_data,dtype=torch.float32,device=device)
val_src_tensor = torch.tensor(val_src_data,dtype=torch.float32,device=device)
#val_src_tensor = val_src_data.clone().detach().to(device)

val_tgt_tensor = torch.tensor(val_tgt_data,dtype=torch.float32,device=device)

# 设置参数
'''
num_epochs = 1000
# 设置优化器和损失函数
optimizer = optim.Adam(model.parameters(),lr=1e-5)
criterion = nn.L1Loss()
# 训练模型
train_model(model, optimizer, criterion, train_src_tensor, train_tgt_tensor, val_src_tensor, val_tgt_tensor, num_epochs=num_epochs, batch_size=128)
'''


'\nnum_epochs = 1000\n# 设置优化器和损失函数\noptimizer = optim.Adam(model.parameters(),lr=1e-5)\ncriterion = nn.L1Loss()\n# 训练模型\ntrain_model(model, optimizer, criterion, train_src_tensor, train_tgt_tensor, val_src_tensor, val_tgt_tensor, num_epochs=num_epochs, batch_size=128)\n'

In [2]:
def inverse_transform_last_column(X, scaler):
    # 获取原始数据的均值和标准差
    mean = scaler.mean_[-1]  # 最后一列的均值
    scale = scaler.scale_[-1]  # 最后一列的标准差

    # 创建一个副本,避免修改原始数据
    X_inv = X.copy()

    # 对最后一列进行反归一化
    X_inv = X_inv * scale + mean

    return X_inv

In [14]:
model.load_state_dict(torch.load('checkpoints/model_epoch_600.pt'))
#model.load_state_dict(torch.load('640_1e-5.pth'))
Y_test_inverse = inverse_transform_last_column(Y_test, scaler)

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model.eval()
model.to('cpu')  # 将模型移动到CPU
with torch.no_grad():
    Y_pred = model(val_src_tensor)
    Y_pred = Y_pred.cpu().numpy().flatten()
    Y_pred_inverse = inverse_transform_last_column(Y_pred, scaler)
    

    mse = mean_squared_error(Y_test_inverse, Y_pred_inverse)
    mae = mean_absolute_error(Y_test_inverse, Y_pred_inverse)
    r2 = r2_score(Y_test_inverse, Y_pred_inverse)
    da = directional_accuracy(Y_test_inverse, Y_pred_inverse)
    print("Mean Squared Error (MSE): ", mse)
    print("Mean Absolute Error (MAE): ", mae)
    print("R-squared (R^2): ", r2)
    print("DA:",da)

Mean Squared Error (MSE):  0.00048017531372350914
Mean Absolute Error (MAE):  0.013786783969065028
R-squared (R^2):  0.1873341248143362
DA: 0.7007074395253309


In [ ]:

model.to('cpu')
model(train_src_tensor.to('cpu'))

model.eval()
model.to('cpu')  # 将模型移动到CPU

with torch.no_grad():
    train_outputs = model(train_src.to('cpu'))  # 将输入数据移动到CPU
    val_outputs = model(val_src.to('cpu'))  # 将输入数据移动到CPU
    
    train_loss = criterion(train_outputs.view(-1), train_tgt.view(-1).to('cpu'))  # 将目标数据移动到CPU
    val_loss = criterion(val_outputs.view(-1), val_tgt.view(-1).to('cpu'))  # 将目标数据移动到CPU

    train_mae = torch.mean(torch.abs(train_tgt.to('cpu') - train_outputs.view(-1)))  # 将目标数据移动到CPU
    val_mae = torch.mean(torch.abs(val_tgt.to('cpu') - val_outputs.view(-1)))  # 将目标数据移动到CPU
    train_rmse = torch.sqrt(torch.mean((train_tgt.to('cpu') - train_outputs.view(-1))**2))  # 将目标数据移动到CPU
    val_rmse = torch.sqrt(torch.mean((val_tgt.to('cpu') - val_outputs.view(-1))**2))  # 将目标数据移动到CPU
    train_da = directional_accuracy(train_tgt.to('cpu'), train_outputs.view(-1))  # 将目标数据移动到CPU
    val_da = directional_accuracy(val_tgt.to('cpu'), val_outputs.view(-1))  # 将目标数据移动到CPU

model.to(device)


